In [1]:
from tdc import Oracle
from vina import Vina
import pyscreener as ps
import os
import numpy as np
import pandas as pd
import csv
import json
import ray
ray.init()

2023-04-01 07:17:54,892	INFO worker.py:1550 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.7.16
Ray version:,2.3.0
Dashboard:,http://127.0.0.1:8265


In [2]:
with open ('/home/pku/BiDe/sele/sampled_sele.json', 'r') as f:
    smiles_dict = json.load(f)

In [3]:
smiles_list = list(smiles_dict.values())

In [7]:
receptor_pdb_list=['7ENE','7L0D','7L10','7M04','7M90','7RFS','7RVT','7S6X','6LU7']


In [6]:
# for i in receptor_pdb_list:
#     if i == '6LU7':
#         pass
#     else:
#         receptor_command = f'prepare_receptor -r /home/pku/3cl/gett_pdb/{i}/{i}_mol2/{i}_new.pdb -o /home/pku/3cl/gett_pdb/docking_receptor/{i}_receptor.pdbqt'
#         os.system(receptor_command)

In [8]:
def save_list_to_csv(filename, data_list):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        for item in data_list:
            csv_writer.writerow([item])

In [9]:
def receptor_box(entry_id = '7ENE'):
    with open(f'./{entry_id}/{entry_id}_result/{entry_id}_new_cavity_1.pdb') as f:
        line_reslut = f.readlines()
        del line_reslut[-1:]
    x_box = []
    y_box = []
    z_box = []
    for j in line_reslut:
        x_box.append(float(j[30:38].strip()))
        y_box.append(float(j[38:46].strip()))
        z_box.append(float(j[46:54].strip()))
    x_max= max(x_box)
    x_min= min(x_box)
    y_max= max(y_box)
    y_min= min(y_box)
    z_max= max(z_box)
    z_min= min(z_box)
    center = [(x_max+x_min)/2,(y_max+y_min)/2,(z_max+z_min)/2]
    return center
    

In [10]:
def docking(entry_id = '7ENE'):
    box = receptor_box(entry_id)
    md = ps.build_metadata("vina", dict(exhaustivness=32))
    vs = ps.virtual_screen(
    "vina",
    receptors=[f'./docking_receptor/{entry_id}_receptor.pdbqt'], 
    center=box,
    size=[35, 35, 35],
    metadata_template=md,
    ncpu=6
    )
    scores = vs(test_id)
    print(entry_id, scores)
    # save_list_to_csv(f'./gett_pdb/docking_receptor/{entry_id}_docking_result.txt', scores)
    # print(scores)
    return scores


In [13]:
test_id = ['CC1([C@@H]2[C@H]1[C@H](N(C2)C(=O)[C@H](C(C)(C)C)NC(=O)NC(C)(C)C)C(=O)N[C@@H](C[C@@H]3CCNC3=O)[C@H](C(=O)N)O)C']

In [14]:
for i in receptor_pdb_list:
    docking(i)

Docking: 100%|██████████| 1/1 [03:00<00:00, 180.02s/ligand]


7ENE [-4.9]


Docking: 100%|██████████| 1/1 [02:56<00:00, 176.11s/ligand]


7L0D [-5.2]


Docking: 100%|██████████| 1/1 [02:54<00:00, 174.86s/ligand]


...

7RVT [-5.4]


Docking: 100%|██████████| 1/1 [02:52<00:00, 172.96s/ligand]


7S6X [-5.9]


Docking: 100%|██████████| 1/1 [03:00<00:00, 180.58s/ligand]


6LU7 [-5.]


In [ ]:
# 7ENE: 7ENE [-6.1], 7L0D [-5.9], 7L10 [-6.2], 7M04 [-6.7], 7M90 [-6.], 7RFS [-5.8], 7RVT [-5.9], 7S6X [-6.4], 6LU7 [-5.9]
# 7L0D: 7L0D [-5.1]
# 7L10: 7L10 [-6.9]
# 7M04: 7M04 [-6.6][-6.6](异构体)
# 7M90: 7M90 [-6.9]
# 7RFS: 7ENE [-5.5], 7L0D [-5.5], 7L10 [-5.6], 7M04 [-5.8], 7M90 [-5.6], 7RFS [-5.2], 7RVT [-5.6], 7S6X [-5.7], 6LU7 [-5]
# 使用7RFS的rcsb中4WI_sdf文件docking[-5.8]
# 7RVT: 7ENE [-5.6], 7L0D [-5.9], 7L10 [-5.7], 7M04 [-6.], 7M90 [-5.5], 7RFS [-5.6], 7RVT [-5.9], 7S6X [-5.6], 6LU7 [-5.4]
# 7S6X: 7ENE [-4.9], 7L0D [-5.2], 7L10 [-4.3], 7M04 [-5.7], 7M90 [-5.3], 7RFS [-5.5], 7RVT [-5.4], 7S6X [-5.9], 6LU7 [-5.9]



In [ ]:
# (base) root@Touk0:/home/test/3cl# vina --receptor ./gett_pdb/docking_receptor/7RFS_receptor.pdbqt --ligand 7rfs_ligand.pdbqt \
#        --config 7rfs_receptor_vina_box.txt \
#        --exhaustiveness=32 --out 7RFS_ligand_vina_out.pdbqt
# #################################################################
# # If you used AutoDock Vina in your work, please cite:          #
# #                                                               #
# # O. Trott, A. J. Olson,                                        #
# # AutoDock Vina: improving the speed and accuracy of docking    #
# # with a new scoring function, efficient optimization and       #
# # multithreading, Journal of Computational Chemistry 31 (2010)  #
# # 455-461                                                       #
# #                                                               #
# # DOI 10.1002/jcc.21334                                         #
# #                                                               #
# # Please see http://vina.scripps.edu for more information.      #
# #################################################################

# Detected 16 CPUs
# Reading input ... done.
# Setting up the scoring function ... done.
# Analyzing the binding site ... done.
# Using random seed: -1982009087
# Performing search ... 
# 0%   10   20   30   40   50   60   70   80   90   100%
# |----|----|----|----|----|----|----|----|----|----|
# ***************************************************
# done.
# Refining results ... done.

# mode |   affinity | dist from best mode
#      | (kcal/mol) | rmsd l.b.| rmsd u.b.
# -----+------------+----------+----------
#    1         -5.8      0.000      0.000
#    2         -5.3      2.960      6.420
#    3         -4.7      3.891      8.226
#    4         -4.6      3.567      6.783
#    5         -4.5      4.146      8.230
#    6         -4.4     10.565     12.369
#    7         -4.4      3.650      7.023
#    8         -4.4      5.080      9.966
#    9         -4.4     11.349     13.173
# Writing output ... done.

In [4]:
receptor_box('7RFS')

[9.2485, -1.2175000000000002, 21.947499999999998]